In [ ]:
#get the parent folder on which this notebook is located
import os
import sys

parent_folder = os.getcwd().replace("\\notebooks","")

#add the parent folder to the system path   
if parent_folder not in sys.path:
    sys.path.append(parent_folder)

In [ ]:
os.environ["APP_CONFIGURATION_URI"] = "https://appconfig-YOURURL.azconfig.io"

#from dotenv import load_dotenv
from connectors import CosmosDBClient
from configuration import Configuration

config = Configuration()
cosmos_client = CosmosDBClient(config)

#load_dotenv()

In [ ]:
async def save_prompt(name:str, content:str):
    #content = content.replace("\n", "\\n")
    item = {
        "id": name,
        "system_prompt": content
    }

    await cosmos_client.upsert_document("prompts", item)

def decode_prompt(name:str):
    item = cosmos_client.get_document("prompts", name)
    if item is None:
        return None
    prompt = item.get("system_prompt", None)
    if prompt is None:
        return None
    
    prompt = prompt.replace("\\n", "\n")

    return prompt

In [ ]:
process_blob = decode_prompt('process_blob')
print(process_blob)

In [ ]:
process_blob = """
Use the 'process_blob' function to process the file: {blob_name}
"""

await save_prompt("process_blob", process_blob)